Playing ATARI-ram games using python-neat and convolutional autoencoder

# Import

In [1]:
from time import time, clock
import numpy as np
import gym
from gym import wrappers
from __future__ import print_function
import os
%matplotlib inline
import neat
import visualize
os.environ['DISPLAY']=':0'

from six.moves import cPickle as pickle
import gym
import numpy as np
import cv2
import multiprocessing
from multiprocessing import Pool
from multiprocessing.reduction import reduce_connection
from hashlib import sha256
import SharedArray as sa

import os, sys, urllib, gzip
from __future__ import print_function
import cPickle as pickle
sys.setrecursionlimit(10000)

from functools import partial

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from IPython.display import Image as IPImage
from PIL import Image
from time import sleep

# Define config

In [3]:
fc_config_filename = 'fc.config'
game_name = 'Skiing'
game_version = 'v0'
game = game_name + '-' + game_version
num_evaluations = 2
num_cores = 11
population_size = 60
encoder_filename = '../collect/{}-enc.pkl'.format(game)
encoder_outputs = 32
cuda_processes = 4

# Create environment

In [4]:
env = gym.make(game)

[2017-02-10 14:27:36,886] Making new env: Skiing-v0


# Autoencoder

In [5]:
def get_ae(encoder_filename):
    from nolearn.lasagne import NeuralNet
    ae = pickle.load(open(encoder_filename,'rb'))
    return(ae)

In [6]:
def get_f_dense(ae):
    import theano
    from lasagne.layers import get_output
    dense_layer = get_output(ae.layers_['encode'],deterministic=True)
    input_var = ae.layers_['input0'].input_var
    f_dense = theano.function([input_var], dense_layer)
    return(f_dense)

In [7]:
def autoencoder(f_dense, picture):
    return(f_dense(np.array([picture], dtype=np.float32))[0])

In [8]:
f_dense = None
get_ae_fn = partial(get_ae, encoder_filename = encoder_filename)
#ae = get_ae_fn()

# Cuda processes

In [9]:
def compress_pipe(p):
    pp = pickle.dumps(reduce_connection(p))
    return(pp)
def decompress_pipe(pp):
    upw = pickle.loads(pp)
    pp = upw[0](upw[1][0],upw[1][1],upw[1][2])
    return(pp)

In [10]:
m = multiprocessing.Manager()
cuda_q = m.Queue()
cuda_r = []

In [11]:
def cuda_process():
    sys.stdout = open(str(os.getpid()) + ".out", "w")
    print("importing...")
    global f_dense
    ae = get_ae_fn()
    f_dense = get_f_dense(ae)
    
    print("Listening")
    global cuda_q
    while True:
        [sn_frame, sn_features, p] = cuda_q.get(block = True)
        fr = sa.attach(sn_frame)
        ft = sa.attach(sn_features)
        p = decompress_pipe(p)
        
        try:
            ft[:] = autoencoder(f_dense, fr)
        except:
            ft[:] = np.zeros(encoder_outputs)

        try:
            p.send(' ')
        except:
            continue
    return(0)

In [12]:
for i in range(cuda_processes):
    cuda_process_p = multiprocessing.Process(target = cuda_process, args = ())
    cuda_process_p.daemon = True
    cuda_process_p.start()
    cuda_r.append(cuda_process_p)
    sleep(1)

Using gpu device 1: GeForce GTX 980 (CNMeM is disabled, cuDNN 5105)
/home/etoestja/venv/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
/home/etoestja/venv/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
INFO (theano.gof.compilelock): Waiting for existing lock by process '28009' (I am process '28023')
[2017-02-10 14:27:57,800] Waiting for existing lock by process '28009' (I am process '28023')
INFO (theano.gof.compileloc

In [13]:
print(cuda_r)

[<Process(Process-2, started daemon)>, <Process(Process-3, started daemon)>, <Process(Process-4, started daemon)>, <Process(Process-5, started daemon)>]


# Create neat-python population

In [14]:
# Load configuration.
config_initial = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                             neat.DefaultSpeciesSet, neat.DefaultStagnation, fc_config_filename)

config_initial.genome_config.num_inputs = encoder_outputs
config_initial.genome_config.num_outputs = env.action_space.n
config_initial.pop_size = population_size

In [15]:
game_fc_config_filename = 'fc-' + game + '.config'

config_initial.save(game_fc_config_filename)

config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation, game_fc_config_filename)

# Create the population, which is the top-level object for a NEAT run.
p = neat.Population(config)

# Add reporters

In [16]:
# Add a stdout reporter to show progress in the terminal.
p.add_reporter(neat.StdOutReporter())
stats = neat.StatisticsReporter()
p.add_reporter(stats)
p.add_reporter(neat.Checkpointer(5))

# Define fitness via game score

In [20]:
alpha = 0.6
sz = 28
f_dense = None

shared_name_frame = None
shared_name_features = None
shared_array_frame = None
shared_array_features = None

def init_shared(obj):
    global shared_name_frame, shared_name_features, shared_array_frame, shared_array_features
    h = sha256(obj.__str__())
    seed = np.frombuffer(h.digest(), dtype='uint32')
    rstate = np.random.RandomState(seed)
    
    shared_name_frame = "shm://" + str(os.getpid()) + '_frame' + str(seed[0])
    shared_name_features = "shm://" + str(os.getpid()) + '_features' + str(seed[0])

    try:
        sa.delete(shared_name_frame[6:])
    except:
        pass
    try:
        sa.delete(shared_name_features[6:])
    except:
        pass
    
    shared_array_frame = sa.create(shared_name_frame, (3, sz, sz))
    shared_array_features = sa.create(shared_name_features, (encoder_outputs))

def transform_observation(observation):
    #return(autoencoder(f_dense, observation))
    global shared_array_frame, shared_name_frame, shared_name_features
    global cuda_q
    a, b = multiprocessing.Pipe()
    b = compress_pipe(b)
    shared_array_frame[:] = np.array(observation * 1./255)
    cuda_q.put([shared_name_frame, shared_name_features, b])
    a.recv()
    return(shared_array_features)

# a = argmax_a Q(s,a)
def predict_action(observation, network):
    observation = transform_observation(observation)
    output = network.activate(observation)
    action = np.argmax(output)
    return(action)

# play num_evaluations games, take mean
def evaluate_network(env, network):
    rewards = []
    i = 0
    while i < num_evaluations:
        rewards += [get_reward(env, network)]
        i += 1

    res = np.array(rewards).mean()
    return res

In [21]:
delta = None

# play 1 game with network
def get_reward(env, network):
    global sz, alpha, delta
    observation = env.reset()
    
    observation = cv2.resize(observation, (sz, sz))
    prev_observation = observation
    delta = np.zeros((3, sz, sz))
    
    done = False
    iteration, total_reward = 0, 0
        
    while not done:
        #env.render()
        action = predict_action(delta, network)
        prev_observation = observation
        observation, reward, done, info = env.step(action)
        observation = cv2.resize(observation, (sz, sz))
        delta_now = observation - prev_observation
        delta_now = np.swapaxes(delta_now, 0, 2)
        delta_now = np.swapaxes(delta_now, 1, 2)
        delta = alpha * delta_now + (1 - alpha) * delta
        
        total_reward += reward

        #if iteration % 10 == 0:
            #plt.imshow(np.swapaxes(delta, 0, 2))
        
        #if iteration % 500 == 0:
        #    print(str(iteration))

        if total_reward < -12000 or iteration >= 8000:
            break

        iteration += 1
        
    return total_reward

def evaluate_genome(genome, config):
    init_shared(genome)
    network = neat.nn.FeedForwardNetwork.create(genome, config)
    fitness = evaluate_network(env, network)
    return fitness

evaluator = neat.parallel.ParallelEvaluator(num_workers = num_cores, eval_function = evaluate_genome, timeout = None)

In [22]:
print(evaluate_genome(p.species.get_species(1).members[1], config))

-12003.0


# Run evolution

In [23]:
# Run evolution
winner = p.run(evaluator.evaluate, 1)


 ****** Running generation 0 ****** 

Population's average fitness: -11999.05000 stdev: 904.34014
Best fitness: -9013.00000 - size: (4, 128) - species 1 - id 16
Species length: 1 totaling 60 individuals
Species no improv: {1: 0}
Average adjusted fitness: 0.490
Spawn amounts: [60]
Species fitness  : [0.48986930896045111]
Mean genetic distance 1.1372074859, std dev 0.202307703154
Total extinctions: 0
Generation time: 310.348 sec
Saving checkpoint to neat-checkpoint-0


In [ ]:
# Run evolution
winner = p.run(evaluator.evaluate, 30)


 ****** Running generation 1 ****** 

Population's average fitness: -11824.87500 stdev: 768.38560
Best fitness: -9012.00000 - size: (4, 128) - species 1 - id 22
Species length: 1 totaling 60 individuals
Species no improv: {1: 0}
Average adjusted fitness: 0.336
Spawn amounts: [60]
Species fitness  : [0.33635130352719123]
Mean genetic distance 1.59431990539, std dev 0.318139914092
Total extinctions: 0
Generation time: 325.387 sec (317.868 average)
Saving checkpoint to neat-checkpoint-1

 ****** Running generation 2 ****** 

Population's average fitness: -11851.54167 stdev: 950.02313
Best fitness: -9013.00000 - size: (4, 128) - species 1 - id 16
Species length: 2 totaling 60 individuals
Species no improv: {1: 1, 2: 0}
Average adjusted fitness: 0.536
Spawn amounts: [53, 7]
Species fitness  : [0.49486092062473042, 0.57638098573054364]
Mean genetic distance 1.57309493696, std dev 0.213539979557
Total extinctions: 0
Generation time: 303.129 sec (312.955 average)
Saving checkpoint to neat-che

# Print results

In [ ]:
# Display the winning genome.
#print('\nBest genome:\n{!s}'.format(winner))

# Show output of the most fit genome against training data.
winner_network = neat.nn.FeedForwardNetwork.create(winner, config)
visualize.draw_net(config, winner, False)
visualize.plot_stats(stats, ylog = False, view = False)
visualize.plot_species(stats, view = False)

#p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-4')
#p.run(eval_genomes, 10)

# Evaluate from checkpoint & send to OpenAI

In [ ]:
def get_winner(p):
    max_fitness = -9999999999
    best_genome = None
    for v in p.population:
        genome = p.population[v]
        if genome.fitness > max_fitness:
            max_fitness = genome.fitness
            winner = genome
    print(max_fitness)
    return(winner)

In [ ]:
s = 'neat-checkpoint-0 neat-checkpoint-15 neat-checkpoint-21 neat-checkpoint-28 neat-checkpoint-34 neat-checkpoint-40 neat-checkpoint-47 neat-checkpoint-53 neat-checkpoint-8 neat-checkpoint-1 neat-checkpoint-16 neat-checkpoint-22 neat-checkpoint-29 neat-checkpoint-35 neat-checkpoint-41 neat-checkpoint-48 neat-checkpoint-54 neat-checkpoint-9 neat-checkpoint-10 neat-checkpoint-17 neat-checkpoint-23 neat-checkpoint-3 neat-checkpoint-36 neat-checkpoint-42 neat-checkpoint-49 neat-checkpoint-55 neat-checkpoint-11 neat-checkpoint-18 neat-checkpoint-24 neat-checkpoint-30 neat-checkpoint-37 neat-checkpoint-43 neat-checkpoint-5 neat-checkpoint-56 neat-checkpoint-12 neat-checkpoint-19 neat-checkpoint-25 neat-checkpoint-31 neat-checkpoint-38 neat-checkpoint-44 neat-checkpoint-50 neat-checkpoint-57 neat-checkpoint-13 neat-checkpoint-2 neat-checkpoint-26 neat-checkpoint-32 neat-checkpoint-39 neat-checkpoint-45 neat-checkpoint-51 neat-checkpoint-6 neat-checkpoint-14 neat-checkpoint-20 neat-checkpoint-27 neat-checkpoint-33 neat-checkpoint-4 neat-checkpoint-46 neat-checkpoint-7 '
for x in s.split(' '):
    p = neat.Checkpointer.restore_checkpoint(x)
    winner = get_winner(p)

In [ ]:
env_eval = gym.make(game)
monitor_path = '/tmp/' + game + '-eval'
env_eval = wrappers.Monitor(env_eval, monitor_path)
def evaluate_with_video(game, network):
    for i_episode in range(100):
        observation = env_eval.reset()
        total_reward = 0
        t = 0
        while True:
            env_eval.render()
            action = predict_action(observation, network)
            observation, reward, done, info = env_eval.step(action)
            total_reward += reward
            t += 1
            if done:
                print("Episode finished after {0} timesteps reward = {1}".format(t+1, total_reward))
                break

In [ ]:
evaluate_with_video(game, winner_network)

In [ ]:
env_eval.close()

In [ ]:
gym.upload(monitor_path, api_key='sk_ciz2F0csRzCkpESayoRuug')

# Measure performance

In [ ]:
# FPS total
t_initial = time()
env.reset()
F = 100
buf = []
for i in range(F):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    features = transform_observation(1)
t_end = time()
print("FPS: " + str(1. * F / (t_end - t_initial)))

In [ ]:
# FPS emulator-only
t_initial = time()
env.reset()
F = 100
buf = []
for i in range(F):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
t_end = time()
print("FPS: " + str(1. * F / (t_end - t_initial)))

In [ ]:
# FPS OD-only
t_initial = time()
env.reset()
F = 100
buf = []
for i in range(F):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    image = process_image(env.ale.getScreenGrayscale()[:, :, 0])
    buf = [image,image,image]
t_end = time()
print("FPS: " + str(1. * F / (t_end - t_initial)))

In [ ]:
init_shared('sdsf')

In [ ]:
for i in range(10):
    get_features_gpu(buf)

In [ ]:
%timeit get_features_gpu(buf)

In [ ]:
def get_reward(env):
    global sz, alpha, delta
    observation = env.reset()
    
    observation = cv2.resize(observation, (sz, sz))
    prev_observation = observation
    delta = np.zeros((3, sz, sz))
    
    done = False
    iteration, total_reward = 0, 0
        
    while not done:
        #env.render()
        #action = predict_action(delta, network)
        print(transform_observation(delta))
        prev_observation = observation
        observation, reward, done, info = env.step(env.action_space.sample())
        observation = cv2.resize(observation, (sz, sz))
        delta_now = observation - prev_observation
        delta_now = np.swapaxes(delta_now, 0, 2)
        delta_now = np.swapaxes(delta_now, 1, 2)
        delta = alpha * delta_now + (1 - alpha) * delta
        
        total_reward += reward

        #if iteration % 10 == 0:
            #plt.imshow(np.swapaxes(delta, 0, 2))
        
        #if iteration % 500 == 0:
        #    print(str(iteration))

        if total_reward < -12000 or iteration >= 8000:
            break

        iteration += 1
        
    return total_reward

In [ ]:
get_reward()